In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis

In [2]:
import pathlib
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
from tensorflow.keras import regularizers

In [4]:
def convert_entire_DF_to_numeric(DF):
    for col in DF.columns:
        DF[col] = pd.to_numeric(DF[col], errors='coerce')
    return DF

def getTheLocalMinMax(df_col1):
    '''
    Code that computes takes the index of the dataframe where the values are either local minima or local maxima
    Parameter: df_col1 is a series whose index is datetime
    Returns a dictionary where the loc_min and loc_max are stored.
    '''
    
    df_col1 = pd.to_numeric(df_col1, errors='coerce')
    loc_min = (np.diff(np.sign(np.diff(df_col1))) > 0).nonzero()[0] + 1         # local min
    loc_max = (np.diff(np.sign(np.diff(df_col1))) < 0).nonzero()[0] + 1         # local max
    dict_ = {'loc_min': loc_min, 'loc_max': loc_max}

    return dict_

def computeInterval_peak_ave(pH_loc_min_series):
    '''
    Code that computes the duration between local minimum of the pH trend.
    Returns the average of the duration in minutes.
    
    Parameter: pH_loc_min_series is the pH reading corresponding to the local minima whose index is the DATETIME
    '''
    ser_TK03_pH_A = pH_loc_min_series
    time_diff_between_peaks = np.diff(ser_TK03_pH_A.index) # get the difference between the peaks in the pH reading series from Pi
    time_diff_bw_peaks_minutes = time_diff_between_peaks/np.timedelta64(1, 'm') # convert the units from nanoseconds to minutes
    
    return time_diff_bw_peaks_minutes.mean()

In [40]:
#load the Pi data

raw_Pi_df = pd.read_excel(r'C:\Users\v.t.flores\Documents\105TK05_TemP_Pi_2020.xlsx') #Load the Pidata from the excel file
raw_Pi_df = raw_Pi_df.set_index('DATETIME')
raw_Pi_df = convert_entire_DF_to_numeric(raw_Pi_df)[1:] # select the all the rows except the first


In [41]:
#filepath 2016 Laboratory Files

filepath_01_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\01 JANUARY 2020\1) JANUARY_2020.xlsx'
filepath_02_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\02 FEBRUARY 2020\2) FEBRUARY _2020.xlsx'
filepath_03_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\03 MARCH 2020\3) March _2020.xlsx'
filepath_04_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\04 APRIL 2020\4) April _2020.xlsx'
filepath_05_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\05 MAY 2020\5) MAY_2020.xlsx'
filepath_06_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\06 JUNE 2020\6) June _2020.xlsx'
filepath_07_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\07 JULY 2020\7) July _2020.xlsx'
filepath_08_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\08 AUGUST 2020\8) August _2020.xlsx'
filepath_09_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\09 SEPTEMBER 2020\9) September _2020.xlsx'


filepath_10_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\10 OCTOBER 2020\10) October _2020.xlsx'
filepath_11_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\11 NOVEMBER 2020\11) November _2020.xlsx'
#filepath_12_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\23) December_2020.xlsx'


currentmonthyr_01_2020 = datetime.date(2020,1,1)
currentmonthyr_02_2020 = datetime.date(2020,2,1)
currentmonthyr_03_2020 = datetime.date(2020,3,1)
currentmonthyr_04_2020 = datetime.date(2020,4,1)
currentmonthyr_05_2020 = datetime.date(2020,5,1)
currentmonthyr_06_2020 = datetime.date(2020,6,1)
currentmonthyr_07_2020 = datetime.date(2020,7,1)
currentmonthyr_08_2020 = datetime.date(2020,8,1)
currentmonthyr_09_2020 = datetime.date(2020,9,1)
currentmonthyr_10_2020 = datetime.date(2020,10,1)
currentmonthyr_11_2020 = datetime.date(2020,11,1)
#currentmonthyr_12_2020 = datetime.date(2020,12,1)

dateList2020 = [currentmonthyr_01_2020,currentmonthyr_02_2020,currentmonthyr_03_2020,currentmonthyr_04_2020,currentmonthyr_05_2020,currentmonthyr_06_2020,
                    currentmonthyr_07_2020,currentmonthyr_08_2020,currentmonthyr_09_2020,currentmonthyr_10_2020,currentmonthyr_11_2020
                    ]

filepathList2020 = [filepath_01_2020,filepath_02_2020,filepath_03_2020,filepath_04_2020,filepath_05_2020,filepath_06_2020,
                    filepath_07_2020,filepath_08_2020,filepath_09_2020,filepath_10_2020,filepath_11_2020,
                    ]

In [42]:
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------

In [43]:
class LabResults():
    def __init__(self, monthyear, filePath):
        #____monthyear format examle is datetime.date(2020,7,1)
        #____filepath example is '\\thgobna001\userdata\THPAL\...\ANALYSIS RESULTS 2020\7) July _2020.xlsx'
        #____add r' at the start of the filepath to apply the regex that allows the use of the filepath as raw
        #____sef.workbook is the uploaded excel workbook of the entire analysis files so it takes time to load
        #____Create an instance of the workbook by giving it a varible name with the sample format: Jan_2020
        
        import openpyxl
        import io
        with open(filePath, "rb") as f:
            in_mem_file = io.BytesIO(f.read())
        
        self.workbook = openpyxl.load_workbook(in_mem_file, data_only=True)
        self.monthyear = monthyear   
        
    def generate_df_from_analysis(self, tabname, list_lab_index):
        #____tabname is a string type name of the tab in the laboratory analysis excel file.
        #____-------Example tabnames: '104PU01', '105TK03', '106TK01'
        #____-------Inside these tabs are the specific analysis results done on the sample taken from the 'tabname' area .
        #____list_lab_index is a list with the sample format is: ['B', 'C', 'E', '104_Pb']
        #____-------the first letter 'B' is the column letter where the day of the month is written as integers 1,2,3..30
        #____-------the second letter 'C' is the column letter where the sampling time is written; e.g. '7:00:00 AM'
        #____-------the third letter 'E' is the column where the results of the specific analysis are listed
        #____-------the last item in the list is the string you'll use as the new title; e.g. '104_Pb'
        print('.', end='')
        workbookTab = self.workbook[tabname]
        self.list_lab_index = list_lab_index
        col_sampleDate = self.list_lab_index[0]
        col_sampleTime = self.list_lab_index[1]
        col_analysis = self.list_lab_index[2]
        col_title = self.list_lab_index[3]
        currentmonthyr = self.monthyear
        
        #get the start and end sample dates
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
        #gets the date day from the B column of the analysis file if there is value in the cell
        for i in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
                Smpl_day_index.append(i)
        #get the ending row that has date day
        for j in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
                smpl_end_index = j
        for y in range(0, len(Smpl_day_index)):
            if Smpl_day_index[y] != Smpl_day_index[-1]:      
                for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                    if workbookTab[str(col_analysis)+str(x)].value is None: 
                        pass
                    else:
                        try:
                            if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                                date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))  
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                            else:
                                date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))  
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                        except:
                            pass

            if Smpl_day_index[y] == Smpl_day_index[-1]:
                for x in range(Smpl_day_index[y], smpl_end_index+1):
                    if workbookTab[str(col_analysis)+str(x)].value is None:   
                        pass
                    else:
                        try:
                            if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                                date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                            else:
                                date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                        except:
                            pass

        Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
        Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
        Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
        Chem_Element_df.set_index('DATETIME', inplace=True)
        print('.', end='')

        return Chem_Element_df

In [44]:
raw_Pi_df = raw_Pi_df[['TK05_Temp','MS_Flow_A','MS_Flow_B']]

In [47]:
raw_Pi_df['MS_Flow'] = np.where(raw_Pi_df['MS_Flow_A']>raw_Pi_df['MS_Flow_B'],\
                                    raw_Pi_df['MS_Flow_A'], raw_Pi_df['MS_Flow_B']) 

DF_Pi = raw_Pi_df

In [16]:
#GATHER LAB RESULTS
MStk01_zdrop = ['B', 'C', 'BD', 'ZN_DROP']

blank_data_2 = {'DATETIME': np.nan, 'ZN_DROP': np.nan}
columnNames = ['DATETIME', 'ZN_DROP']


df_zdrop = pd.DataFrame(blank_data_2, columns = columnNames)
df_zdrop = df_zdrop.set_index('DATETIME')

#for i in range(len(dateList2020)):
for i in range(len(dateList2020)):
    
    monthLab = LabResults(dateList2020[i], filepathList2020[i])
    df_month = monthLab.generate_df_from_analysis('106TK01', MStk01_zdrop)
    print(i)
    df_zdrop = df_zdrop.append(df_month, sort=True)


#-------------------------------------

..0
..1
..2
..3
..4
..5
..6
..7
..8
..9
..10


In [ ]:
DF_Pi = DF_Pi.drop(['MS_Flow_A','MS_Flow_B'], axis=1)

In [57]:
DF_main = DF_Pi.join(df_zdrop, how='outer')
DF_main = convert_entire_DF_to_numeric(DF_main) # select the all the rows except the first
DF_main = DF_main.dropna()
DF_main['ZN_DROP'] = DF_main['ZN_DROP']*100

#### QUERYING

In [60]:
DF_main[DF_main['TK05_Temp']>75].loc['2020-01-01':'2020-02-01']
#DF_main[DF_main['']<70]

,TK05_Temp,MS_Flow,ZN_DROP
DATETIME,,,
2020-01-01 17:00:00,75.130138,851.238202,71.232877
2020-01-01 18:00:00,75.095427,857.243375,79.166667
2020-01-01 21:00:00,75.050932,901.355539,26.923077
2020-01-02 19:00:00,75.060947,813.263723,55.882353
2020-01-06 01:00:00,75.747532,671.690777,75.675676
2020-01-06 03:00:00,75.293421,548.756273,56.862745
2020-01-06 04:00:00,75.937018,548.767912,73.529412
2020-01-06 17:00:00,75.879205,701.585993,73.404255
2020-01-06 22:00:00,76.137655,650.265646,73.333333


In [65]:
DF_main.loc['2020-01-06':'2020-01-08'][50:]

,TK05_Temp,MS_Flow,ZN_DROP
DATETIME,,,
2020-01-08 02:00:00,74.412180,650.703615,78.571429
2020-01-08 03:00:00,74.204132,649.455285,58.333333
2020-01-08 04:00:00,75.052190,649.714343,47.619048
2020-01-08 05:00:00,75.525405,651.580307,76.785714
2020-01-08 06:00:00,74.560335,765.620589,81.707317
2020-01-08 07:00:00,72.803239,799.659858,73.584906
2020-01-08 08:00:00,72.620945,811.657257,57.692308
2020-01-08 09:00:00,73.122424,902.672966,72.881356
2020-01-08 10:00:00,73.617906,899.295945,74.074074


In [113]:
#tk14 +3hrs: TK14_air_Flow', 'TK14_Acid_Flow', 'SPT_4_inch_FR', 'SPT_3_inch_FR, 106TK14_pH, FNTRL_SLUR_DENSITY_1
#106VE04_Ni + 2hrs : 106VE04_Ni
#108 + 3hrs: CCD9_UF	CCD8_UF	FNTRL_SR_AT	FNTRL_SLUR_DENSITY_2

#tempor.sort_values(by='FNTRL_SR_AT_max', ascending=False)

#### Adjusting the datetime to compensate for the retention time

In [203]:
tk14_cols_list = ['TK14_air_Flow', 'TK14_Acid_Flow', 'SPT_4_inch_FR', 'SPT_3_inch_FR', '106TK14_pH', '106TK14_ORP', 'FNTRL_SLUR_DENSITY']
ve04_cols_list = ['106VE04_Ni']
fntrl_cols_list = ['CCD9_UF', 'CCD8_UF', 'FNTRL_SR_AT', 'FNTRL_SR_AT_max','FNTRL_SLUR_DENSITY']
ms_cols_list = ['MS_FeedLineTemp', 'H2S_flow_to_MS', 'MS_Flow','NaSH_flow', '106TK01_Ni', '106TK01_pH']

#adjust the datetime of the items related to tk14. add the retention time
tk14_df_adj = DF_main[tk14_cols_list].reset_index()
tk14_df_adj['DATETIME'] = tk14_df_adj['DATETIME'] - datetime.timedelta(hours=3)
tk14_df_adj = tk14_df_adj.set_index('DATETIME')
tk14_df_adj.rename(columns={'FNTRL_SLUR_DENSITY':'FNTRL_SLUR_DENSITY_1'}, inplace=True) #rename the fntrl slur density

#adjust the datetime of the items related to ve04. add the retention time
ve04_df_adj = DF_main[ve04_cols_list].reset_index()
ve04_df_adj['DATETIME'] = ve04_df_adj['DATETIME'] - datetime.timedelta(hours=2)
ve04_df_adj = ve04_df_adj.set_index('DATETIME')

#adjust the datetime of the items related to tk14. add the retention time
fntrl_df_adj = DF_main[fntrl_cols_list].reset_index()
fntrl_df_adj['DATETIME'] = fntrl_df_adj['DATETIME'] - datetime.timedelta(hours=4)
fntrl_df_adj = fntrl_df_adj.set_index('DATETIME')
fntrl_df_adj.rename(columns={'FNTRL_SLUR_DENSITY':'FNTRL_SLUR_DENSITY_2'}, inplace=True) #rename the fntrl slur density

#store the items related to ms to ms_df_adj, though we didnt adjst it for RT
ms_df_adj = DF_main[ms_cols_list]

#Combine all the 'sub dfs'
DF_main_RT_adj = ms_df_adj.join(fntrl_df_adj, how='outer').join(ve04_df_adj, how='outer').join(tk14_df_adj, how='outer')
DF_main_RT_adj = DF_main_RT_adj.dropna(how='any')

#### Preprocessing for tensorflow

In [209]:
column_names_for_tensorflow = ['TK14_air_Flow', 'TK14_Acid_Flow', 'SPT_4_inch_FR', 'SPT_3_inch_FR', 'FNTRL_SLUR_DENSITY_1',
 'CCD9_UF', 'CCD8_UF','FNTRL_SR_AT_max','FNTRL_SLUR_DENSITY_2','MS_FeedLineTemp', 
 'H2S_flow_to_MS','MS_Flow','NaSH_flow', '106TK01_Ni', '106TK01_pH']
DF_for_tensorflow = DF_main_RT_adj[column_names_for_tensorflow]
DF_for_tensorflow = DF_for_tensorflow[DF_for_tensorflow['MS_Flow']>100]

In [217]:
dataset2 = DF_for_tensorflow
train_dataset2 = dataset2.sample(frac=0.8, random_state=0)
test_dataset2 = dataset2.drop(train_dataset2.index)


# Also look at the overall statistics:
train_stats = train_dataset2.describe()
train_stats.pop('FNTRL_SR_AT_max')
train_stats = train_stats.transpose()

#Separate the target value, or "label", from the features. 
#This label is the value that you will train the model to predict.

train_labels = train_dataset2.pop('FNTRL_SR_AT_max')
test_labels = test_dataset2.pop('FNTRL_SR_AT_max')

def norm(x):
    return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset2)
normed_test_data = norm(test_dataset2)
#normed_train_labels = norm(train_labels)

In [ ]:
def norm(x, df_stat):
    return (x - df_stat['mean']) / df_stat['std']
#normed_Zdrop_data = norm(DF_TF_Zdrop, Zdrop_data_stats)

In [225]:
# testing
# The patience parameter is the amount of epochs to check for improvement
EPOCHS = 1000
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001, mode='auto', verbose=0)

model = keras.Sequential([layers.Dense(512, activation=tf.nn.relu, 
                                       kernel_regularizer=regularizers.l2(0.0001), 
                                       input_shape=[len(train_dataset2.keys())]),
                          layers.Dropout(0.5),
                          layers.Dense(512, activation=tf.nn.relu,
                                       kernel_regularizer=regularizers.l2(0.0001)),
                          layers.Dropout(0.5),
                          layers.Dense(512, activation=tf.nn.relu,
                                       kernel_regularizer=regularizers.l2(0.0001)),        
                          layers.Dropout(0.5),
                          layers.Dense(1)])

#optimizer = tf.keras.optimizers.RMSprop(0.001)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

model.compile(loss='mse',optimizer=optimizer,metrics=['mae', 'mse', 'accuracy'])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001, mode='auto', verbose=1)
model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                   validation_split = 0.2, verbose=1, callbacks=[early_stop])



Train on 4225 samples, validate on 1057 samples
Epoch 1/1000
4225/4225 [==============================] - 2s 440us/sample - loss: 9.0802 - mean_absolute_error: 1.7546 - mean_squared_error: 8.9744 - acc: 7.1006e-04 - val_loss: 6.9284 - val_mean_absolute_error: 1.5916 - val_mean_squared_error: 6.8210 - val_acc: 9.4607e-04
Epoch 2/1000
4225/4225 [==============================] - 1s 191us/sample - loss: 8.4425 - mean_absolute_error: 1.7001 - mean_squared_error: 8.3336 - acc: 7.1006e-04 - val_loss: 6.9546 - val_mean_absolute_error: 1.4691 - val_mean_squared_error: 6.8444 - val_acc: 9.4607e-04
Epoch 3/1000
4225/4225 [==============================] - 1s 195us/sample - loss: 8.3388 - mean_absolute_error: 1.6631 - mean_squared_error: 8.2271 - acc: 4.7337e-04 - val_loss: 6.7233 - val_mean_absolute_error: 1.5981 - val_mean_squared_error: 6.6104 - val_acc: 9.4607e-04
Epoch 4/1000
4225/4225 [==============================] - 1s 196us/sample - loss: 8.0512 - mean_absolute_error: 1.6379 - mean_squa

Epoch 31/1000
4225/4225 [==============================] - 1s 196us/sample - loss: 4.8675 - mean_absolute_error: 1.3581 - mean_squared_error: 4.6746 - acc: 7.1006e-04 - val_loss: 5.6704 - val_mean_absolute_error: 1.3317 - val_mean_squared_error: 5.4759 - val_acc: 9.4607e-04
Epoch 32/1000
4225/4225 [==============================] - 1s 200us/sample - loss: 5.0975 - mean_absolute_error: 1.3508 - mean_squared_error: 4.9016 - acc: 7.1006e-04 - val_loss: 6.2763 - val_mean_absolute_error: 1.3173 - val_mean_squared_error: 6.0793 - val_acc: 9.4607e-04
Epoch 33/1000
4225/4225 [==============================] - 1s 184us/sample - loss: 5.1926 - mean_absolute_error: 1.3368 - mean_squared_error: 4.9945 - acc: 7.1006e-04 - val_loss: 5.0753 - val_mean_absolute_error: 1.3449 - val_mean_squared_error: 4.8761 - val_acc: 9.4607e-04
Epoch 34/1000
4225/4225 [==============================] - 1s 197us/sample - loss: 5.6096 - mean_absolute_error: 1.3486 - mean_squared_error: 5.4091 - acc: 7.1006e-04 - val_lo

NameError: name 'model_Zdrop' is not defined

In [230]:
test_predictions = model.predict(normed_test_data).flatten()

In [231]:
normed_test_data

,TK14_air_Flow,TK14_Acid_Flow,SPT_4_inch_FR,SPT_3_inch_FR,FNTRL_SLUR_DENSITY_1,CCD9_UF,CCD8_UF,FNTRL_SLUR_DENSITY_2,MS_FeedLineTemp,H2S_flow_to_MS,MS_Flow,NaSH_flow,106TK01_Ni,106TK01_pH
DATETIME,,,,,,,,,,,,,,
2020-01-01 07:00:00,0.784719,-0.106801,-0.292205,-0.331552,-0.933698,-0.866935,-0.939049,-1.088763,0.192915,-0.947316,-0.929248,-1.357032,-0.110366,-0.235319
2020-01-01 14:00:00,0.798789,-0.097734,-0.513019,-0.338861,-0.560990,-0.765149,-1.056173,-0.592358,-0.178270,-1.322846,-1.339241,-1.604617,-0.480077,-0.372730
2020-01-02 04:00:00,0.490316,-0.091176,-0.455848,-0.336574,-0.818881,-1.016504,-1.237181,-0.734858,0.210586,-1.253066,-0.885985,-1.344803,-0.541724,-0.464338
2020-01-02 07:00:00,0.728462,-0.112075,-3.835552,3.149071,-0.986274,-1.016352,-1.236972,-1.074250,0.226761,-1.134414,-0.882850,-1.346466,-1.200517,-0.418534
2020-01-02 08:00:00,0.680198,-0.109224,-3.831776,3.183611,-1.073498,-1.014300,-1.237152,-1.199052,0.192706,-1.074453,-0.889592,-1.343996,-0.809038,-0.311659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-15 12:00:00,0.423323,-0.102053,0.300604,3.487971,0.282532,-0.070474,-0.049779,0.286261,0.224255,0.051800,0.600237,0.587082,-0.868422,-1.563627
2020-11-15 15:00:00,0.471623,-0.100531,0.146230,3.355984,0.654350,0.523853,0.188271,0.813274,-0.143382,0.265054,0.550002,0.761958,-0.855709,-0.235319
2020-11-15 16:00:00,0.494819,-0.098919,0.395926,3.547789,0.811771,0.529788,0.308290,0.867602,-0.234560,0.309075,0.518281,0.761320,-0.875387,-0.143712


In [264]:
def norm(x, df_stat):
    return (x - df_stat['mean']) / df_stat['std']
#normed_Zdrop_data = norm(DF_TF_Zdrop, Zdrop_data_stats)
DF_for_tensorflow_stats = DF_for_tensorflow.describe()
DF_for_tensorflow_stats = DF_for_tensorflow_stats.transpose()

normed_data = norm(DF_for_tensorflow ,DF_for_tensorflow_stats)
normed_data_feats = norm(DF_for_tensorflow ,DF_for_tensorflow_stats)
normed_data_feats.pop('FNTRL_SR_AT_max')

gastec_predictions = model.predict(normed_data_feats).flatten()
#gastec_predictions.shape

x_act = DF_for_tensorflow['FNTRL_SR_AT_max'].index
y_act = DF_for_tensorflow['FNTRL_SR_AT_max']
x_pred = DF_for_tensorflow.index
y_pred = gastec_predictions


fig = make_subplots(
    rows=1, cols=1,
    specs=[[{"type": "scatter"}]])

fig.add_trace(go.Scatter(x=x_act, y=y_act, name='108_SR_act'), secondary_y=False,) #This code adds the TF_pred2 to the plot
fig.add_trace(go.Scatter(x=x_pred, y=y_pred, name='108_SR_pred'), secondary_y=False,) #This code adds the TF_pred2 to the plot
#fig.add_trace(go.Scatter(x=DF_TF_Zdrop.index, y=DF_TF_Zdrop['Zn_106_pred'], name='Zn_106_pred'), secondary_y=False,) #This code adds the TF_pred2 to the plot

DATETIME
2020-01-01 07:00:00   -0.374461
2020-01-01 08:00:00   -0.339779
2020-01-01 09:00:00   -0.308428
2020-01-01 10:00:00   -0.718172
2020-01-01 11:00:00   -0.723251
                         ...   
2020-11-16 05:00:00    0.729477
2020-11-16 06:00:00    0.690535
2020-11-16 07:00:00   -0.625048
2020-11-16 08:00:00   -0.670763
2020-11-16 09:00:00   -0.670763
Name: FNTRL_SR_AT_max, Length: 6603, dtype: float64

,TK14_air_Flow,TK14_Acid_Flow,SPT_4_inch_FR,SPT_3_inch_FR,FNTRL_SLUR_DENSITY_1,CCD9_UF,CCD8_UF,FNTRL_SR_AT_max,FNTRL_SLUR_DENSITY_2,MS_FeedLineTemp,H2S_flow_to_MS,MS_Flow,NaSH_flow,106TK01_Ni,106TK01_pH
DATETIME,,,,,,,,,,,,,,,
2020-01-01 07:00:00,2751.503773,0.002585,63.778001,0.018481,1105.289830,289.788692,320.086360,1.115000,1095.323031,73.151327,1240.623288,984.710700,1.539782,3.31300,3.1200
2020-01-01 08:00:00,2758.750520,0.001720,62.722772,-0.051095,1095.323031,294.233386,312.204872,1.217417,1094.738310,73.181101,1180.027987,899.828734,1.419610,3.25280,3.1125
2020-01-01 09:00:00,2744.838200,0.002588,60.111889,-0.038346,1094.738310,309.826253,300.087609,1.310000,1107.508336,73.093333,1180.097208,899.176193,1.421227,3.19260,3.1050
2020-01-01 10:00:00,2743.527983,0.003271,60.722319,-0.032522,1107.508336,309.829682,307.990517,0.100000,1118.228451,73.069419,1180.292679,900.746711,1.419936,3.18230,3.0975
2020-01-01 11:00:00,2759.458489,0.004063,60.415173,-0.003990,1118.228451,310.248365,319.970471,0.085000,1125.659355,73.068114,1178.731291,899.212119,1.419246,3.17200,3.0900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-16 05:00:00,2563.666788,0.002174,75.521626,-0.309853,1253.645016,503.677340,409.924662,4.375000,1233.752923,73.393866,2052.886083,1600.300672,3.488471,3.20010,3.0700
2020-11-16 06:00:00,2519.426026,0.002981,75.151710,-0.040147,1233.752923,500.166812,422.957443,4.260000,1213.303243,73.457275,2083.568908,1599.190127,3.487077,3.24405,3.0950
2020-11-16 07:00:00,2537.530725,0.002381,73.629030,-0.050037,1213.303243,466.960007,414.744723,0.375000,1203.979612,73.636304,2056.822409,1599.715066,3.489191,3.28800,3.1200


In [272]:
DF_for_tensorflow_2 = DF_for_tensorflow

In [273]:
DF_for_tensorflow_2.iloc[0:1].pop('FNTRL_SR_AT_max')

DATETIME
2020-01-01 07:00:00    1.115
Name: FNTRL_SR_AT_max, dtype: float64

In [292]:
test_values = DF_for_tensorflow_2.drop('FNTRL_SR_AT_max', axis=1).iloc[500:510]
test_values_1 = test_values.reset_index().drop('DATETIME', axis=1)

In [307]:
test_values_1['TK14_Acid_Flow'] = 0.002094
test_values_1['SPT_4_inch_FR'] = 65
test_values_1['106TK01_Ni'] = 3.5
test_values_1['106TK01_pH'] = 3.1
test_values_1['CCD9_UF'] = 650
test_values_1['CCD8_UF'] = 600
test_values_1['FNTRL_SLUR_DENSITY_2'] = 1160
test_values_1['FNTRL_SLUR_DENSITY_1'] = 1160
test_values_1['SPT_3_inch_FR'] = 0
test_values_1['NaSH_flow'] = np.linspace(1.5,9, 10)
test_values_1

,TK14_air_Flow,TK14_Acid_Flow,SPT_4_inch_FR,SPT_3_inch_FR,FNTRL_SLUR_DENSITY_1,CCD9_UF,CCD8_UF,FNTRL_SLUR_DENSITY_2,MS_FeedLineTemp,H2S_flow_to_MS,MS_Flow,NaSH_flow,106TK01_Ni,106TK01_pH
0,2448.255602,0.002094,65,0,1160,650,600,1160,73.304819,2287.796324,1421.046520,1.500000,3.5,3.1
1,2448.255602,0.002094,65,0,1160,650,600,1160,73.540623,2265.472131,1399.844042,2.333333,3.5,3.1
2,2448.255602,0.002094,65,0,1160,650,600,1160,73.619928,2269.984235,1400.404138,3.166667,3.5,3.1
3,2448.255602,0.002094,65,0,1160,650,600,1160,73.549742,2258.891548,1432.588489,4.000000,3.5,3.1
4,2448.255602,0.002094,65,0,1160,650,600,1160,73.472491,2245.584378,1496.550265,4.833333,3.5,3.1
5,2448.255602,0.002094,65,0,1160,650,600,1160,73.575452,2298.320320,1466.475929,5.666667,3.5,3.1
6,2448.255602,0.002094,65,0,1160,650,600,1160,73.504093,2243.585360,1478.304092,6.500000,3.5,3.1
7,2448.255602,0.002094,65,0,1160,650,600,1160,73.451331,2243.105670,1486.335796,7.333333,3.5,3.1
8,2448.255602,0.002094,65,0,1160,650,600,1160,73.405198,2250.948071,1489.271561,8.166667,3.5,3.1
9,2448.255602,0.002094,65,0,1160,650,600,1160,73.349652,2252.292208,1491.099171,9.000000,3.5,3.1


In [306]:
def norm(x, df_stat):
    return (x - df_stat['mean']) / df_stat['std']
#normed_Zdrop_data = norm(DF_TF_Zdrop, Zdrop_data_stats)

In [331]:
test_values_2 = test_values_1[['TK14_air_Flow', 'TK14_Acid_Flow', 'SPT_4_inch_FR', 'SPT_3_inch_FR',
       'FNTRL_SLUR_DENSITY_1', 'CCD9_UF', 'CCD8_UF', 'FNTRL_SLUR_DENSITY_2',
       'MS_FeedLineTemp', 'H2S_flow_to_MS', 'MS_Flow', 'NaSH_flow',
       '106TK01_Ni', '106TK01_pH']]
#df_stat_test = test_values_1.describe().transpose()
normed_data = norm(test_values_2, DF_for_tensorflow_stats.drop('FNTRL_SR_AT_max'))
#use the tk05_Zn_model to predict the tk05 Zn concentration

test_values_1['predicted_gastec'] = model.predict(normed_data)
#test_values_1

In [332]:
test_values_1['predicted_gastec']

0    5.457313
1    4.245577
2    3.688535
3    2.554707
4    1.652003
5    1.150496
6    0.967697
7    0.888144
8    0.836195
9    0.802584
Name: predicted_gastec, dtype: float32

In [334]:
test_values_1.to_excel(r'C:\Users\v.t.flores\Documents\108_test.xlsx')

In [313]:
normed_data

,TK14_air_Flow,TK14_Acid_Flow,SPT_4_inch_FR,SPT_3_inch_FR,FNTRL_SLUR_DENSITY_1,CCD9_UF,CCD8_UF,FNTRL_SLUR_DENSITY_2,MS_FeedLineTemp,H2S_flow_to_MS,MS_Flow,NaSH_flow,106TK01_Ni,106TK01_pH
0,-0.948683,0.948683,NaN,NaN,NaN,NaN,NaN,NaN,-1.707290,1.378557,-0.906626,-1.486301,NaN,-0.948683
1,-0.948683,0.948683,NaN,NaN,NaN,NaN,NaN,NaN,0.626354,0.203856,-1.453573,-1.156012,NaN,-0.948683
2,-0.948683,0.948683,NaN,NaN,NaN,NaN,NaN,NaN,1.411193,0.441283,-1.439125,-0.825723,NaN,-0.948683
3,-0.948683,0.948683,NaN,NaN,NaN,NaN,NaN,NaN,0.716598,-0.142415,-0.608885,-0.495434,NaN,-0.948683
4,-0.948683,0.948683,NaN,NaN,NaN,NaN,NaN,NaN,-0.047922,-0.842639,1.041097,-0.165145,NaN,-0.948683
5,-0.948683,0.948683,NaN,NaN,NaN,NaN,NaN,NaN,0.971039,1.932330,0.265288,0.165145,NaN,-0.948683
6,-0.948683,0.948683,NaN,NaN,NaN,NaN,NaN,NaN,0.264833,-0.947828,0.570412,0.495434,NaN,-0.948683
7,-0.948683,0.948683,NaN,NaN,NaN,NaN,NaN,NaN,-0.257329,-0.973069,0.777601,0.825723,NaN,-0.948683
8,-0.948683,0.948683,NaN,NaN,NaN,NaN,NaN,NaN,-0.713881,-0.560401,0.853333,1.156012,NaN,-0.948683
9,-0.948683,0.948683,NaN,NaN,NaN,NaN,NaN,NaN,-1.263595,-0.489673,0.900479,1.486301,NaN,-0.948683


In [340]:
nash_max = DF_for_tensorflow['NaSH_flow'].max()
DF_for_tensorflow[DF_for_tensorflow['NaSH_flow'] > 5]

,TK14_air_Flow,TK14_Acid_Flow,SPT_4_inch_FR,SPT_3_inch_FR,FNTRL_SLUR_DENSITY_1,CCD9_UF,CCD8_UF,FNTRL_SR_AT_max,FNTRL_SLUR_DENSITY_2,MS_FeedLineTemp,H2S_flow_to_MS,MS_Flow,NaSH_flow,106TK01_Ni,106TK01_pH
DATETIME,,,,,,,,,,,,,,,
2020-01-19 14:00:00,2474.749553,0.002425,71.336104,-0.035109,1143.816624,520.030116,520.489133,0.625333,1135.821011,73.194583,2227.859900,1448.803215,5.279852,3.973950,3.152500
2020-01-21 00:00:00,2514.437808,0.002538,75.015462,-0.045613,1149.813343,629.698390,630.152296,0.490000,1153.376469,73.602258,2187.464589,1500.586560,5.470568,3.693500,3.180000
2020-01-21 01:00:00,2516.565221,0.002027,71.197397,-0.033106,1153.376469,630.059684,593.344207,0.760000,1159.481328,73.526364,2171.818439,1476.104238,5.461256,3.648000,3.190000
2020-01-22 08:00:00,2408.896916,0.001719,74.688936,-0.039955,1159.009486,650.148322,619.718543,3.235000,1145.110899,73.575452,2298.320320,1466.475929,5.875582,3.735750,3.110000
2020-04-16 09:00:00,2431.031168,0.927803,57.266165,27.204906,1216.850305,331.131774,327.485535,3.945000,1204.196263,70.041224,691.525447,774.551774,5.004679,2.446100,3.057500
2020-04-16 10:00:00,2552.300818,0.998965,64.325503,29.533456,1204.196263,299.713214,274.974114,0.945000,1160.936017,71.246884,634.173558,800.500232,5.001926,2.430550,3.088750
2020-05-29 01:00:00,2500.251986,0.002929,76.768571,-0.021295,1202.151662,580.279045,591.182209,0.695000,1203.963195,73.598166,2352.655900,1482.010950,5.141896,3.792100,3.165000
2020-10-07 18:00:00,2723.652356,0.490778,58.177266,27.894377,1246.944748,0.759565,249.689422,0.030000,1238.471436,67.213244,0.759583,247.065203,9.366664,3.162143,3.203750
2020-10-23 04:00:00,2569.886446,1.799941,73.123242,35.438642,999.659983,205.120262,-0.156495,29.215000,999.643860,64.089756,0.696057,387.832560,8.879574,3.337215,2.812500
